In [1]:
import pandas as pd
import os
import numpy as np
import time
from datetime import datetime
to_datetime = lambda d: pd.to_datetime(d, format='%d/%m/%Y')

In [26]:
def chimp_finder():
    
    dir = '../cotacoes/dados/b3/cotacoes_diarias/'

    # arquivo = 'QUAL3.sa.csv'

    ## Inicio
    inicio = time.perf_counter()
    micos = pd.DataFrame()
    x = y = 0

    ## Varrer os arquivos no diretório
    for arquivo in os.listdir(dir):

        ## Lê os arquivos
        cotacoes = pd.read_csv(dir + arquivo, parse_dates=['date'], date_parser=to_datetime).sort_values(by='date', ascending=True)

        ## Cálculos
        cotacoes['simbolo'] = arquivo[0:-7]
        cotacoes['vol_med'] = cotacoes['vol'].rolling(window=5).mean()      ## Média Móvel do Volume
        cotacoes['var_vol'] = cotacoes['vol']/cotacoes['vol'].shift(1)        ## Variação do Volume no dia
        cotacoes['var_med'] = cotacoes['vol']/cotacoes['vol_med']             ## Variação do Volume em relação a Média Móvel
        cotacoes['saida'] = cotacoes['open'].shift(-1)                        ## Saída da Estratégia
        cotacoes['lucro'] = cotacoes['saida'] - cotacoes['close']             ## Lucro da Estrategia
        cotacoes['var_lucro'] = (cotacoes['lucro']/cotacoes['close']) * 100   ## Variação % do Lucro
        cotacoes['var_close'] = (cotacoes['saida'] / cotacoes['saida'].shift(1) - 1) * 100 ## Variação % do Fechamento

        ## Seleção
        selecao = (cotacoes['var_vol'] >= 0) & (cotacoes['var_med'] >= 3) ## Variação diária ou relativa a média > 10

        if len(micos) == 0:
            micos = cotacoes[selecao].copy()
        else:
            micos = micos.append(cotacoes[selecao])

        ## Contadores
        x += 1
        y += len(cotacoes)

    micos.set_index(['simbolo','date'], inplace=True)

    ## Fim
    fim = time.perf_counter()
    
    return micos

In [27]:
micos = chimp_finder()
print(len(micos))
micos.head(10)

30243


open    high     low   close    neg          vol  \
simbolo date                                                             
OULG11B 2019-05-22  105.88  105.88  103.00  103.00     33     51300292   
        2019-05-23  103.21  103.21  102.83  102.83     25    109488904   
        2019-06-07  104.51  105.98  103.50  104.00     25     33742897   
        2019-07-05  114.00  114.00  110.00  110.00     37     10070500   
        2019-07-16  111.00  113.13  109.01  111.00     12     48263766   
        2019-07-25  114.49  115.98  114.49  115.98     12     16393151   
        2019-08-06  121.99  125.00  120.00  120.00     20     43975631   
        2019-08-14  114.20  118.00  114.20  118.00      8     35381109   
LIGT3   2011-11-30   26.61   27.70   26.60   27.41  13513  21297323300   
        2013-05-31   17.90   18.24   17.27   17.37  10393  13613215200   

                         vol_med      var_vol   var_med   saida  lucro  \
simbolo date                                                             
OULG11B 2019-05-22  1.599556e+07    75.369229  3.207158  103.21   0.21   
        2019-05-23  3.366670e+07     2.134274  3.252143  103.02   0.19   
        2019-06-07  8.453307e+06  1061.097390  3.991680  105.89   1.89   
        2019-07-05  2.688244e+06    25.287515  3.746125  111.49   1.49   
        2019-07-16  1.094379e+07    32.932254  4.410152  112.00   1.00   
        2019-07-25  4.402078e+06    60.751375  3.723957  115.98   0.00   
        2019-08-06  9.026815e+06    81.737268  4.871666  115.00  -5.00   
        2019-08-14  7.825155e+06    14.436555  4.521458  116.02  -1.98   
LIGT3   2011-11-30  5.593927e+09    11.501596  3.807222   27.68   0.27   
        2013-05-31  4.372152e+09     4.063656  3.113619   17.56   0.19   

                    var_lucro  var_close  
simbolo date                              
OULG11B 2019-05-22   0.203883  -2.521723  
        2019-05-23   0.184771  -0.184091  
        2019-06-07   1.817308   1.320448  
        2019-07-05   1.354545  -2.201754  
        2019-07-16   0.900901   0.900901  
        2019-07-25   0.000000   1.301424  
        2019-08-06  -4.166667  -5.729978  
        2019-08-14  -1.677966   1.593695  
LIGT3   2011-11-30   0.985042   4.021045  
        2013-05-31   1.093840  -1.899441